In [1]:
from inference.select_best_models import *
# tar cf best_models.tar best_models
# tar cf saved_models.tar saved_models

/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "CGCNN",
    "interpolation": False,
    "struct_type": "unrelaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████████████████████████████████████████████████| 1252/1252 [00:00<00:00, 4038.21it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|████████████████████████████████████████████████████████| 5064/5064 [02:14<00:00, 37.53it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|█████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 10586.26it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|█████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 10039.15it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [6]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "CGCNN",
    "interpolation": False,
    "struct_type": "relaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████████████████████████████████████████████████| 1252/1252 [00:00<00:00, 3574.29it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|████████████████████████████████████████████████████████| 5064/5064 [02:15<00:00, 37.49it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 9926.22it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|█████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 10203.38it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [7]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "CGCNN",
    "interpolation": False,
    "struct_type": "M3Gnet_relaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████████████████████████████████████████████████| 1252/1252 [00:00<00:00, 4027.35it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|████████████████████████████████████████████████████████| 5064/5064 [02:19<00:00, 36.25it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 9898.02it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|██████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 9708.91it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [2]:
gpu_num=2
target_prop="Op_band_center"

model_params = {
    "model_type": "CGCNN",
    "interpolation": False,
    "struct_type": "unrelaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████████████████████████████████████████████████| 1252/1252 [00:00<00:00, 3976.95it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|████████████████████████████████████████████████████████| 5064/5064 [02:13<00:00, 37.98it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|█████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 10014.76it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|█████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 10906.50it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [3]:
gpu_num=2
target_prop="Op_band_center"

model_params = {
    "model_type": "CGCNN",
    "interpolation": False,
    "struct_type": "relaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████████████████████████████████████████████████| 1252/1252 [00:00<00:00, 3572.64it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|████████████████████████████████████████████████████████| 5064/5064 [02:12<00:00, 38.26it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|█████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 10654.96it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|█████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 10779.58it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [4]:
gpu_num=2
target_prop="Op_band_center"

model_params = {
    "model_type": "CGCNN",
    "interpolation": False,
    "struct_type": "M3Gnet_relaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|██████████████████████████████████████████████████████| 1252/1252 [00:00<00:00, 4056.45it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


100%|████████████████████████████████████████████████████████| 5064/5064 [02:16<00:00, 37.14it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


100%|██████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 9969.55it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


100%|█████████████████████████████████████████████████████| 5064/5064 [00:00<00:00, 10841.77it/s]


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [5]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "e3nn",
    "interpolation": False,
    "struct_type": "unrelaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|████████████████████████████████████████████████████████| 1252/1252 [00:37<00:00, 33.02it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [6]:
gpu_num=2
target_prop="dft_e_hull"
 
model_params = {
    "model_type": "e3nn",
    "interpolation": False,
    "struct_type": "relaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|████████████████████████████████████████████████████████| 1252/1252 [00:37<00:00, 33.63it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [7]:
gpu_num=2
target_prop="dft_e_hull"

model_params = {
    "model_type": "e3nn",
    "interpolation": False,
    "struct_type": "M3Gnet_relaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|████████████████████████████████████████████████████████| 1252/1252 [00:38<00:00, 32.55it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [8]:
gpu_num=2
target_prop="Op_band_center"

model_params = {
    "model_type": "e3nn",
    "interpolation": False,
    "struct_type": "unrelaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|████████████████████████████████████████████████████████| 1252/1252 [00:38<00:00, 32.84it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [9]:
gpu_num=2
target_prop="Op_band_center"

model_params = {
    "model_type": "e3nn",
    "interpolation": False,
    "struct_type": "relaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|████████████████████████████████████████████████████████| 1252/1252 [00:39<00:00, 31.39it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved


In [10]:
gpu_num=2
target_prop="Op_band_center"

model_params = {
    "model_type": "e3nn",
    "interpolation": False,
    "struct_type": "M3Gnet_relaxed",
}

reverify_sigopt_models(model_params, gpu_num, target_prop)
keep_the_best_few_models(model_params, target_prop)

Loaded data
Completed data processing


100%|████████████████████████████████████████████████████████| 1252/1252 [00:40<00:00, 30.97it/s]
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #0


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #1


...

Reverifying sigopt model #48


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "
/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/sigopt/objects.py:342: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(


Reverifying sigopt model #49


/home/jypeng/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


Best models #0 found and saved
Best models #1 found and saved
Best models #2 found and saved
